# THE IMPORTS

In [7]:
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2 import sql
from psycopg2.extras import Json
from pgvector.psycopg2 import register_vector
from openai import OpenAI
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
import hashlib
import uuid
import time
import ollama

# THE FUNCTIONS

In [8]:
# function to directly execute a query
def postgres_execute(conn, query, get_result=False,commit=True,print_query=False):
    if print_query: 
        print(query)
    with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
    #with conn.cursor() as cursor:
        cursor.execute(query)
        if commit:
            conn.commit()
        if get_result:
            return cursor.fetchall()
        else:
            return None

In [9]:
# Function to generate embeddings using OpenAI API
def get_openai_embedding(clientopenAI,text):
    response = clientopenAI.embeddings.create(input=text, model="text-embedding-3-small")
    embedding = response.data[0].embedding
    token_used = response.usage.total_tokens
    result = {}
    result["embedding"] = embedding
    result["token"] = token_used
    return result

def get_ollama_embedding(text,myModel="mistral"):
    #response = clientopenAI.embeddings.create(input=text, model="text-embedding-3-small")
    response = ollama.embeddings(model=myModel, prompt=text)
    embedding = response["embedding"]
    token_used = -1 # Ollama doesn't return the token used
    result = {}
    result["embedding"] = embedding
    result["token"] =token_used #
    return result

# Function to query similar documents
def query_documents(embedding_method,clientopenAI,question, cur, conn, n_results=2, myModel="mistral"):

    # ----------------------------------------------------------------------------------
    # -------------------------- WE EXTRACT THE SUB SET OF CHUNCK TO EMBED -------------
    # we extract the chunks from the database
    start_time = time.time()
    targetChunckTable = ""
    if embedding_method == "OLLAMA":
        #targetChunckTable = "document_embeding_mistral"
        targetChunckTable = "document_embeding_mistral_generic"
        query_embedding = get_ollama_embedding(question,myModel)['embedding']
    elif embedding_method == "OPENAI":
        targetChunckTable = "document_embeding_openai"
        query_embedding = get_openai_embedding(clientopenAI,question)['embedding']
    end_time = time.time()
    print("Time to get the embedding: ", end_time - start_time)
    
    start_time = time.time()
    query = """select 
        d.document_uuid as document_uuid,
        d.document_name as document_name,
        d.document_location as document_location,
        d.document_hash as document_hash,
        d.document_type as document_type,
        d.document_status as document_status,
        dc.document_chunk_uuid as document_chunk_uuid,
        dc.embebed_text as embebed_text,
        emb.document_embeding_uuid as document_embeding_uuid,
        emb.embeder_type as embeder_type,
        emb.embedding_token as embedding_token,
        emb.embedding_time as embedding_time
        from document_library.""" + targetChunckTable + """ emb
        inner join document_library.document_chunks dc on dc.document_chunk_uuid = emb.document_chunk_uuid 
        inner join document_library.documents d on d.document_uuid = dc.document_uuid
        order by embedding <-> %s::vector
        limit %s;
        """
    params = (query_embedding, n_results)
    query = cur.mogrify(query, params).decode('utf-8')
    print(query)
    results = postgres_execute(conn, query, commit=True,get_result=True)
    end_time = time.time()
    print("Time to get the chuncks from the db: ", end_time - start_time)

    return results

# Function to generate response
def generate_response(answer_method,clientopenAI,question, relevant_chunks):
    context = "\n\n".join(relevant_chunks)
    prompt = (
        "You are an assistant for question-answering tasks. Use the retrieved context below "
        "to answer the question. If you don't know, say so in three sentences maximum."
        "\n\nContext:\n" + context + "\n\nQuestion:\n" + question
    )

    if answer_method == "OLLAMA":
        response = ollama.chat(model="mistral",messages=[
        {"role": "system", "content": prompt},
        {"role": "user","content": question},
        ])
        return response['message']['content']
    elif answer_method == "OPENAI":
        response = clientopenAI.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": question},
            ],
        )
        return response.choices[0].message.content

    


# WE LOAD THE ENV VARIABLES AND CREATE THE ENVIRONMENT

In [10]:
# Load environment variables
load_dotenv()


# OpenAI client
openai_key = os.getenv("OPENAI_API_KEY")
client_OPENAI = OpenAI(api_key=openai_key)

# Initialize PostgreSQL connection
conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    password=os.getenv("DB_PASSWORD"),
)

conn.autocommit = True

# WHY????
register_vector(conn)

# DATABASE PREPARATION 

In [11]:
# VECTOR EXTENSION
query = "CREATE EXTENSION IF NOT EXISTS vector;"
postgres_execute(conn, query)

# BE CAREFULL: VERIFY THE COMMIT
# conn.commit()

# WE QUERY THE DATA

In [12]:
question = "Comment Renforcerl'applicationdesméthodologiespourl'ESG?"#"How many people lack access to electricity globally?"
#question = "How many people love PALO IT?"
question = "Comment Renforcerl'applicationdesméthodologiespourl'ESG?"

cur = conn.cursor()

# select the embedding method
embedding_method = "OLLAMA"#"OPENAI"
answer_method = "OPENAI"#"OLLAMA"#"OPENAI"
#targeted_document_uudi = 'f8eb9af1-6648-4da3-b231-9326b46c77ec'

# Query the documents
start_time = time.time()
relevant_chunks_dico = query_documents(embedding_method,client_OPENAI,question,cur,conn,myModel="nomic-embed-text")

print("Relevant Chunks Dico:", relevant_chunks_dico)
end_time = time.time()
query_time = end_time - start_time
print("Query Time Question:", query_time)

 # Generate response   
print("Generating response...")
relevant_chunks = [chunk['embebed_text'] for chunk in relevant_chunks_dico]
print("Relevant Chunks:", relevant_chunks)

start_time = time.time()
answer = generate_response(answer_method,client_OPENAI,question, relevant_chunks)
end_time = time.time()
reponse_time = end_time - start_time
print("Response Generation Time:", reponse_time)
print("Answer:", answer)

Time to get the embedding:  0.6657440662384033
select 
        d.document_uuid as document_uuid,
        d.document_name as document_name,
        d.document_location as document_location,
        d.document_hash as document_hash,
        d.document_type as document_type,
        d.document_status as document_status,
        dc.document_chunk_uuid as document_chunk_uuid,
        dc.embebed_text as embebed_text,
        emb.document_embeding_uuid as document_embeding_uuid,
        emb.embeder_type as embeder_type,
        emb.embedding_token as embedding_token,
        emb.embedding_time as embedding_time
        from document_library.document_embeding_mistral_generic emb
        inner join document_library.document_chunks dc on dc.document_chunk_uuid = emb.document_chunk_uuid 
        inner join document_library.documents d on d.document_uuid = dc.document_uuid
        order by embedding <-> ARRAY[1.0361977815628052,0.5330418348312378, -2.635094165802002, -1.0697836875915527,1.34493565